In [1]:
import lib.custom_callbacks as callbacks
import lib.custom_metrics as metrics
import lib.evaluation as ev
import lib.plotting as plot
import lib.models as models
import lib.dataset as dt

import matplotlib.pyplot as plt
import imgaug.augmenters as iaa
import tensorflow as tf
import pandas as pd
import numpy as np
import datetime
import os

from importlib import reload

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPU


In [3]:
DR_LEVELS_PER_CLASS = [[0], [1,2,3,4]]

IMAGE_SIZE = (540, 540, 3)

# Specify dataset files
TRAIN_FILE = 'DATASET-TRAIN-80.csv'
VALIDATION_FILE = 'DATASET-VALIDATION-10_BALANCED.csv'

ONE_HOT_FORMAT = False

TRAINING_BATCH_SIZE = 12
TRAINING_DATA_AUG = True
TRAINING_BALANCED = False
TRAINING_PREFETCH = 20
TRAINING_TAKE_SIZE = None

VALIDATION_BATCH_SIZE = 12
VALIDATION_DATA_AUG = False
VALIDATION_BALANCED = False
VALIDATION_PREFETCH = 1
VALIDATION_TAKE_SIZE = None

In [4]:
reload(dt)

'''
def create_dataset_new(csv_file, 
                        list_list_classes,
                        balanced=False,
                        apply_data_augmentation=False,
                        batch_size=1, 
                        prefetch_buffer=None, 
                        shuffle=False,
                        size=None):
'''

train_dataset, y_true_train = dt.create_dataset_new(TRAIN_FILE, 
                                                    DR_LEVELS_PER_CLASS, 
                                                    balanced=TRAINING_BALANCED, 
                                                    apply_data_augmentation=TRAINING_DATA_AUG, 
                                                    batch_size=TRAINING_BATCH_SIZE, 
                                                    prefetch_buffer=TRAINING_PREFETCH, 
                                                    one_hot_format=ONE_HOT_FORMAT,
                                                    size=TRAINING_TAKE_SIZE)

val_dataset, y_true_val = dt.create_dataset_new(VALIDATION_FILE, 
                                                DR_LEVELS_PER_CLASS, 
                                                balanced=VALIDATION_BALANCED,
                                                apply_data_augmentation=VALIDATION_DATA_AUG,
                                                batch_size=VALIDATION_BATCH_SIZE,
                                                prefetch_buffer=VALIDATION_PREFETCH, 
                                                is_validation=True,
                                                one_hot_format=ONE_HOT_FORMAT,
                                                size=VALIDATION_TAKE_SIZE)

Se aplica data aug


## Class weights

In [6]:
from sklearn.utils.class_weight import compute_class_weight

class_weights = compute_class_weight('balanced', classes=np.unique(y_true_train[:,0]), y=y_true_train[:,0])
d_class_weights = dict(enumerate(class_weights))
print(d_class_weights)

{0: 0.6740610690645092, 1: 1.9362775165269543}


In [5]:
# Define model
reload(models)

model = models.inceptionResNetV2.get_model(input_shape=(540,540,3), num_outputs=len(DR_LEVELS_PER_CLASS))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, 1536)              54336736  
_________________________________________________________________
flatten (Flatten)            (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 3074      
Total params: 54,339,810
Trainable params: 54,279,266
Non-trainable params: 60,544
_________________________________________________________________


In [7]:
# Load weights if they were saved
base_path = 'saved_weights/inceptionResNetV2/SGD_2/'

# Pre-trained model
model_path = 'saved_weights/inceptionResNetV2/SGD/bestAUC_DRlvls_0_1234.h5'

if not os.path.exists(base_path):
    os.mkdir(base_path)

if os.path.exists(model_path):
    model.load_weights(model_path)
    print('Model loaded')

Model loaded


In [8]:
# Save ground truth for validation dataset
np.save(base_path + 'ground_truth_val.npy', y_true_val)

In [9]:
# Compile model
reload(metrics)

classes_names = [''.join(list(map(str,class_))) for class_ in DR_LEVELS_PER_CLASS]

print(classes_names)

metric_AUC_0_1234 = metrics.Wrapper_AUC(classes=[1], original_dr_lvls=classes_names, is_one_hot=ONE_HOT_FORMAT)

metric_Sp_at_95_Sens_0_1234 = metrics.Wrapper_SpecificityAtSensitivity(sensitivity=0.95, classes=[1], original_dr_lvls=classes_names, is_one_hot=ONE_HOT_FORMAT)

['0', '1234']


In [10]:
cbacks = [tf.keras.callbacks.ModelCheckpoint(base_path + 'best_' + metric_AUC_0_1234.get_custom_name() + '.h5', 
                                             monitor='val_' + metric_AUC_0_1234.get_custom_name(),
                                             save_best_only=True,
                                             save_weights_only=True,
                                             mode='max'),

          tf.keras.callbacks.ModelCheckpoint(base_path + 'best_' + metric_Sp_at_95_Sens_0_1234.get_custom_name() + '.h5', 
                                             monitor='val_' + metric_Sp_at_95_Sens_0_1234.get_custom_name(),
                                             save_best_only=True,
                                             save_weights_only=True,
                                             mode='max'),
          
          tf.keras.callbacks.ModelCheckpoint(base_path + 'best_accuracy.h5',
                                             monitor='val_accuracy',
                                             save_best_only=True,
                                             save_weights_only=True,
                                             mode='max'),

          tf.keras.callbacks.ModelCheckpoint(base_path + 'best_loss.h5',
                                             monitor='val_loss',
                                             save_best_only=True,
                                             save_weights_only=True,
                                             mode='min'),
          
          callbacks.Save_Training_Evolution(base_path + 'training_evolution.csv'),
          
          # learning rate scheduler
          tf.keras.callbacks.LearningRateScheduler(callbacks.create_scheduler_function(10, 0.9))
]

In [11]:
reload(metrics)

validation_dir = base_path+'validation_outputs/'

if not os.path.exists(validation_dir):
    os.mkdir(validation_dir)

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, clipnorm=1.0),
              metrics=['accuracy',
                       metric_AUC_0_1234, # DR levels: 0 vs 1,2,3,4
                       metric_Sp_at_95_Sens_0_1234, # DR levels: 0 vs 1,2,3,4
                       metrics.RunningValidation(path=validation_dir, n_columns=len(DR_LEVELS_PER_CLASS)) # THIS METRIC MUST BE INIZIALIZATED BEFORE EVERY TRAINING
                      ]) 

num_epochs = 2000

# history = model.fit(train_dataset.take(50), epochs=num_epochs, validation_data=val_dataset.take(10), verbose=1, callbacks=cbacks, class_weight=d_class_weights)
history = model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset, verbose=1, callbacks=cbacks, class_weight=d_class_weights)

Epoch 1/2000
4736/4736 [==============================] - 2856s 603ms/step - loss: 0.3815 - accuracy: 0.8693 - AUC_DRlvls_0_1234: 0.8995 - Sp_at_95_sens_DRlvls_0_1234: 0.3972 - RunningValidation: 0.0000e+00 - val_loss: 0.3684 - val_accuracy: 0.8432 - val_AUC_DRlvls_0_1234: 0.9091 - val_Sp_at_95_sens_DRlvls_0_1234: 0.4458 - val_RunningValidation: 1.0000
Epoch 2/2000
4736/4736 [==============================] - 2855s 603ms/step - loss: 0.3847 - accuracy: 0.8663 - AUC_DRlvls_0_1234: 0.8972 - Sp_at_95_sens_DRlvls_0_1234: 0.3933 - RunningValidation: 0.0000e+00 - val_loss: 0.3624 - val_accuracy: 0.8471 - val_AUC_DRlvls_0_1234: 0.9024 - val_Sp_at_95_sens_DRlvls_0_1234: 0.3663 - val_RunningValidation: 1.0000
Epoch 3/2000
4736/4736 [==============================] - 2861s 604ms/step - loss: 0.3868 - accuracy: 0.8658 - AUC_DRlvls_0_1234: 0.8964 - Sp_at_95_sens_DRlvls_0_1234: 0.3803 - RunningValidation: 0.0000e+00 - val_loss: 0.3882 - val_accuracy: 0.8463 - val_AUC_DRlvls_0_1234: 0.9011 - val_Sp_

KeyboardInterrupt: 

In [12]:
model.load_weights(base_path + 'best_loss.h5')

In [13]:
model.compile(loss='sparse_categorical_crossentropy', 
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, clipnorm=1.0),
              metrics=['accuracy',
                       metric_AUC_0_1234, # DR levels: 0 vs 1,2,3,4
                       metric_Sp_at_95_Sens_0_1234, # DR levels: 0 vs 1,2,3,4
                      ]) 

In [14]:
validation_pred = model.predict(val_dataset)

In [18]:
model.evaluate(val_dataset)

300/300 [==============================] - 53s 177ms/step - loss: 0.3624 - accuracy: 0.8471 - AUC_DRlvls_0_1234: 0.9024 - Sp_at_95_sens_DRlvls_0_1234: 0.3663


[0.36237919330596924,
 0.8471372723579407,
 0.9024046063423157,
 0.3663146197795868]

In [45]:
np.savetxt(base_path + 'val_no_data_aug.txt', validation_pred)

In [16]:
print(validation_pred.shape)

(3598, 2)


In [19]:
reload(dt)

val_dataset2, y_true_val2 = dt.create_dataset_new(VALIDATION_FILE, 
                                                DR_LEVELS_PER_CLASS, 
                                                balanced=VALIDATION_BALANCED,
                                                apply_data_augmentation=VALIDATION_DATA_AUG,
                                                batch_size=VALIDATION_BATCH_SIZE,
                                                prefetch_buffer=VALIDATION_PREFETCH, 
                                                is_validation=True,
                                                one_hot_format=ONE_HOT_FORMAT,
                                                size=VALIDATION_TAKE_SIZE,
                                                 val_data_aug=True)

Se aplica data aug para val


In [20]:
model.evaluate(val_dataset2)

300/300 [==============================] - 52s 174ms/step - loss: 0.3958 - accuracy: 0.8243 - AUC_DRlvls_0_1234: 0.8880 - Sp_at_95_sens_DRlvls_0_1234: 0.3307


[0.39580607414245605,
 0.8243468403816223,
 0.8879876732826233,
 0.330739289522171]

In [21]:
model.evaluate(val_dataset2)

300/300 [==============================] - 53s 175ms/step - loss: 0.3854 - accuracy: 0.8291 - AUC_DRlvls_0_1234: 0.8931 - Sp_at_95_sens_DRlvls_0_1234: 0.4019


[0.3854046165943146,
 0.8290717005729675,
 0.893149197101593,
 0.40188995003700256]

In [22]:
validation2_preds = [model.predict(val_dataset2) for i in range(5)]

In [50]:
for i in range(5):
    np.savetxt(base_path + 'val_data_aug_conJigsaw_' + str(i) + '.txt', validation2_preds[i])

In [52]:
y_true_val.shape

(3598, 2)

In [56]:
validation2_preds[0].shape

(3598, 2)

In [32]:
reload(dt) # se cambio en el codigo la llamada al data augmentation

val_dataset3, y_true_val3 = dt.create_dataset_new(VALIDATION_FILE, 
                                                DR_LEVELS_PER_CLASS, 
                                                balanced=VALIDATION_BALANCED,
                                                apply_data_augmentation=VALIDATION_DATA_AUG,
                                                batch_size=VALIDATION_BATCH_SIZE,
                                                prefetch_buffer=VALIDATION_PREFETCH, 
                                                is_validation=True,
                                                one_hot_format=ONE_HOT_FORMAT,
                                                size=VALIDATION_TAKE_SIZE,
                                                 val_data_aug=True)

Se aplica data aug para val
sin jigsaw


In [33]:
model.evaluate(val_dataset3)

300/300 [==============================] - 52s 174ms/step - loss: 0.3721 - accuracy: 0.8402 - AUC_DRlvls_0_1234: 0.8987 - Sp_at_95_sens_DRlvls_0_1234: 0.3741


[0.37213635444641113,
 0.8401889801025391,
 0.898659884929657,
 0.37409672141075134]

In [34]:
model.evaluate(val_dataset3)

300/300 [==============================] - 53s 176ms/step - loss: 0.3833 - accuracy: 0.8357 - AUC_DRlvls_0_1234: 0.8934 - Sp_at_95_sens_DRlvls_0_1234: 0.3624


[0.38325247168540955,
 0.8357420563697815,
 0.8933516144752502,
 0.3624235689640045]

In [35]:
validation3_preds = [model.predict(val_dataset3) for i in range(5)]

In [49]:
for i in range(5):
    np.savetxt(base_path + 'val_data_aug_sinJigsaw_' + str(i) + '.txt', validation3_preds[i])